In [43]:
#Script to analyze data from RCP-PCR experiments.
#Run the script in the RCP-PCR directory to analyze test data.
#To analyze own data, prepare a directory with name of data under RCP-PCR/Data with uncompressed R1/R2 fastq files.

In [44]:
import os 
import sys
import numpy as np
import multiprocessing as mp
from Bio import SeqIO
import operator
from pprint import pprint

In [45]:
# Put directory name under 'Data' with fastq files.
dir_name    = 'test'

#Tag file
tag_file    = 'tag_assignment_test.csv'

# Number of threads used to execute commands.
threads     = 2

In [46]:
#Identify the fastq files to create small fasta files for easier handling.


fastq_files = []
PATH = os.path.abspath(".")
lis = os.listdir("%s/Data/%s" % (PATH,dir_name))
for f in lis:
    if f[-6:] == ".fastq" :
        fastq_files.append(f)
print("List of input fastq files: %s \n\n (There should be two files corresponding to R1/R2.)"%(fastq_files))


List of input fastq files: ['test_R1.fastq', 'test_R2.fastq'] 

 (There should be two files corresponding to R1/R2.)


In [47]:
#Creating directries for data proccessing.

try:
    os.makedirs('%s/Data/%s/fragmented_fasta'%(PATH,dir_name))
except OSError:
    pass

try:
    os.makedirs('%s/Data/%s/blast'%(PATH,dir_name))
    os.makedirs('%s/Data/%s/blast/sh.blast'%(PATH,dir_name))
    os.makedirs('%s/Data/%s/blast/out.blast'%(PATH,dir_name))
except OSError:
    pass

try:
    os.makedirs('%s/Data/%s/identification'%(PATH,dir_name))
    os.makedirs('%s/Data/%s/identification/sh.identification'%(PATH,dir_name))
    os.makedirs('%s/Data/%s/identification/out.identification'%(PATH,dir_name))
except OSError:
    pass

try:
    os.makedirs('%s/Data/%s/count'%(PATH,dir_name))
    os.makedirs('%s/Data/%s/count/sh.count'%(PATH,dir_name))
    os.makedirs('%s/Data/%s/count/out.count'%(PATH,dir_name))
except OSError:
    pass

In [48]:
#Defining function to create small fasta files.
def make_fasta_from_listoftups(L,name):
    with open("%s"%name,"w") as f:
        for I in L:
            #print I
            f.write('>%s'%(str(I[0])))
            f.write('%s\n'%(str(I[1])))
        f.close()
        print("Made fna file : %s"%(name))

def make_fragmented_fasta(F,split_num,read_length,out_dir):
    line_c  = 0
    read_c  = 0
    file_count   = 0
    with open(F,"r") as f:
        fasta_tup = []
        for line in f:
            #print(line)
            if (line_c % 4 == 0):
                read_ID = line.replace(" ","_").replace(":","_").replace("@","")

            elif (line_c % 4 == 1):
                if read_length > len(line):
                    seq = line
                else:
                    seq     = line[:read_length]

            elif (line_c % 4 == 3):
                qscore  = line 
                fasta_tup.append( (read_ID,seq) )
                read_c +=1
                
                if (read_c % split_num == 0):
                    file_count +=1
                    
                    f_name = ("").join([F.split("/")[-1].split(".")[0] , "_%d"%(file_count),".fna"])
                    out    = ("/").join( [out_dir,f_name]) 

                    make_fasta_from_listoftups(fasta_tup, out )
                    
                    fasta_tup = []
            line_c += 1
        f.close()
    pass

In [49]:
#Creating directory for fragmented fasta files and executing the function above. 
for F in fastq_files:
     make_fragmented_fasta("%s/Data/%s/%s"%(PATH,dir_name,F),400000,150,"%s/Data/%s/fragmented_fasta"%(PATH,dir_name)) 

Made fna file : /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/fragmented_fasta/test_R1_1.fna
Made fna file : /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/fragmented_fasta/test_R1_2.fna
Made fna file : /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/fragmented_fasta/test_R1_3.fna
Made fna file : /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/fragmented_fasta/test_R1_4.fna
Made fna file : /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/fragmented_fasta/test_R1_5.fna
Made fna file : /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/fragmented_fasta/test_R1_6.fna
Made fna file : /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/fragmented_fasta/test_R2_1.fna
Made fna file : /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/fragmented_fasta/test_R2_2.fna
Made fna file : /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/fragmented_fasta/test_R2_3.fna
Made fna file : /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/D

In [50]:
#Creating BLAST database.
os.system("makeblastdb -in %s/const-seq.fna -dbtype nucl"%(PATH))
db = '%s/const-seq.fna'%(PATH)

# You should be able to see the following output in your terminal 
#
#Building a new DB, current time: 03/09/2021 13:20:38
#New DB name:   /Users/username/GitHub/BFG-Y2H/RCP-PCR/const-seq.fna
#New DB title:  /Users/username/GitHub/BFG-Y2H/RCP-PCR/const-seq.fna
#Sequence type: Nucleotide
#Keep MBits: T
#Maximum file size: 1000000000B
#Adding sequences from FASTA; added 32 sequences in 0.00142884 seconds.

In [51]:
# Deefining multi-proccessing handling.
def subprocces_sh(core,sh_dir):
    command_L = get_sh(sh_dir)
    command_L = ["sh %s/%s"%(sh_dir,i) for i in command_L]
    if core > len(command_L):
        core = len(command_L)
    n = len(command_L)/core
    pool = mp.Pool(core)
    feed = {}
    for i in range(core):
        feed.update({i:[]})
    lp = len(command_L)
    ky = range(0,lp)
    v = 0
    for i in range(lp):
        feed[v].append(command_L[0])
        del command_L[0]
        v +=1
        if (v==core):
            v = 0
    fed = []
    for i in range(0,core):
        fed.append(feed[i])
    
    results = pool.map(run_sh,fed)
    pass

def get_sh(x):
    files = []
    lis = os.listdir("%s" %x)
    for i in lis:
        if i[-3:] == ".sh" :
            files.append(i)
    return files

def run_sh(sh_L):
    for sh in sh_L:
        print("Running command : %s\n"%(sh))
        os.system(sh)
    
        


In [52]:
# Prepare .sh files for executing BLAST

#Listing fragmented fastas files.
fasta_files = os.listdir("%s/Data/%s/fragmented_fasta" % (PATH,dir_name)) 

#Generating sh files.
sh = 1
for f in fasta_files:
    command = "blastn -task blastn-short -strand plus -db %s/const-seq.fna -outfmt 10 -evalue 1e-3 -query %s/Data/%s/fragmented_fasta/%s -out %s/Data/%s/blast/out.blast/%s.blast"%(PATH,PATH,dir_name,f,PATH,dir_name,f.split(".")[0])
    f_name = "%s/Data/%s/blast/sh.blast/blast_%d.sh"%(PATH,dir_name,sh)
    with open(f_name,"w") as F:
        F.write(command)
        print("Generated %s"%(f_name))
    F.close()
    sh +=1


    

Generated /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/blast/sh.blast/blast_1.sh
Generated /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/blast/sh.blast/blast_2.sh
Generated /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/blast/sh.blast/blast_3.sh
Generated /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/blast/sh.blast/blast_4.sh
Generated /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/blast/sh.blast/blast_5.sh
Generated /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/blast/sh.blast/blast_6.sh
Generated /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/blast/sh.blast/blast_7.sh
Generated /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/blast/sh.blast/blast_8.sh
Generated /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/blast/sh.blast/blast_9.sh
Generated /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/blast/sh.blast/blast_10.sh
Generated /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/bl

In [53]:
# Executing BLAST sh files
print("Dividing and runnig jobs . . . ")
subprocces_sh(threads,"%s/Data/%s/blast/sh.blast"%(PATH,dir_name))
print ("\n\nDone")

Dividing and runnig jobs . . . 
Running command : sh /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/blast/sh.blast/blast_11.sh
Running command : sh /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/blast/sh.blast/blast_4.sh


Running command : sh /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/blast/sh.blast/blast_5.sh

Running command : sh /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/blast/sh.blast/blast_1.sh

Running command : sh /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/blast/sh.blast/blast_8.sh

Running command : sh /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/blast/sh.blast/blast_10.sh

Running command : sh /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/blast/sh.blast/blast_2.sh

Running command : sh /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/blast/sh.blast/blast_9.sh

Running command : sh /Users/danyamamotoevans/GitHub/BFG-Y2H/RCP-PCR/Data/test/blast/sh.blast/blast_6.sh

Running command : sh 

In [54]:
#Reading reference files
bar2num_file = "%s/bar2num.txt"%(PATH)
bar_d = {}
with open(bar2num_file,"r") as F:
    for line in F:
        cols = line.split(",")
        num = cols[0]  
        bar = cols[1]
        bar_d[num] = bar
    F.close()

tag_f    = "%s/%s"%(PATH,tag_file)
tags = []
with open(tag_f,"r") as F:
    for line in F:
        cols = line.split(",")
        #AD003,AD,BC,P01-P01
        plate        = cols[0]
        barcode_type = cols[1] 
        PCR_type     = cols[2] 
        tag          = cols[3] 
        tags.append([plate,barcode_type,PCR_type,tag])
    F.close()    

In [55]:
#Function to parse fasta file to doctionary.
def fasta2dict(name):
    d = {}
    for record in SeqIO.parse(name,"fasta"):
        d.update({record.id :  str(record.seq)})
    return d


#Function to parse BLAST output to dictionary
def parse_blast_output2dict(blast_R1,blast_R2,PATH,dir_name,tag_f): 
    # BLAST database sequences in dict format
    seq_DB   = fasta2dict("%s/const-seq.fna"%(PATH))
    
    
    blast_d = {}
    
    with open(blast_R1) as F:
        for line in F:
            cols           = line.split("\n")[0].split(",")
            #print(cols)
            read           = cols[0]   
            subject        = cols[1]  
            str_on_read    = int(cols[6])  
            end_on_read    = int(cols[7])
            str_on_subject = int(cols[8])
            end_on_subject = int(cols[9])
            evalue         = float(cols[10])
            
           
            if (str_on_read < end_on_read):
                if (str_on_subject ==1):
                    if (end_on_subject==len(seq_DB[subject])):
                        element = {"str":str_on_read,
                                   "end":end_on_read,
                                   "evalue":evalue}
                        try:
                            blast_d[read]["R1"][subject] = element
                        except KeyError:
                            try:
                                blast_d[read]["R1"]       = {}
                                blast_d[read]["R1"][subject] = element
                            except KeyError:
                                blast_d[read] = {}
                                blast_d[read]["R1"]       = {}
                                blast_d[read]["R1"][subject] = element
 
        F.close()
    #pprint(blast_d)  
    with open(blast_R2) as F:
        for line in F:
            cols             = line.split(",")
            read             = cols[0]   
            subject          = cols[1]   
            str_on_read      = int(cols[6]   )
            end_on_read      = int(cols[7]   )
            str_on_subject   = int(cols[8]   )
            end_on_subject   = int(cols[9]   )
            evalue           = float(cols[10])  
    
            if ((str_on_read < end_on_read) and (str_on_subject ==1) and (end_on_subject==len(seq_DB[subject]))):
                element = {"str":str_on_read,
                               "end":end_on_read,
                               "evalue":evalue}
                try:
                    blast_d[read]["R2"][subject] = element
                except KeyError:
                    try:
                        blast_d[read]["R2"] = {}
                        blast_d[read]["R2"][subject] = element
                    except KeyError:
                        blast_d[read] = {}
                        blast_d[read]["R2"] = {}
                        blast_d[read]["R2"][subject] = element
                
        F.close()
        
    
    return blast_d         

In [56]:
# Function to assign RCP-PCR category and extract index sequences. 
def assign_category(read_d,read_R1_seq,read_R2_seq):

    R1 = read_d['R1']
    R2 = read_d['R2']

    # PS1.0 and PS2.0 at the reasonable positions? (end at <40bp)
    PS_pos = 0

    P_SEQ1 = 0
    P_SEQ2 = 0

    if( ('PS1.0-primer'in R1.keys()) and ('PS2.0-primer' in R2.keys()) ):
        if( (R1['PS1.0-primer']['end_on_read'] < 40) and (R2['PS2.0-primer']['end_on_read'] < 40) ):
            P_SEQ1 = read_R1_seq[R1['PS1.0-primer']['str_on_read']-10:-2]
            P_SEQ2 = read_R2_seq[R2['PS2.0-primer']['str_on_read']-10:-2]
            PS_pos = 1
            
    # Row and Column priming sites from the reasonable positions? (start at <50bp)
    # Which category? DB-BC / DB-lox / AD-BC / AD-lox?
    DB_BC = 0
    DB_lox = 0
    AD_BC = 0
    AD_lox = 0
    
    

    if(PS_pos ==1):
        R_seq = {}
        C_seq = {}
        #DB-BC
        if( ('DBU1-primer' in R1.keys()) and ('DBD2-primer' in R2.keys())):
            if (R1['PS1.0-primer']['end_on_read'] < R1['DBU1-primer']['str_on_read']):
               if (R1['DBU1-primer']['str_on_read'] < 50):
                   if (R2['PS2.0-primer']['end_on_read'] < R2['DBD2-primer']['str_on_read']):
                        if (R2['DBD2-primer']['str_on_read'] < 50):

                            R_seq['DB-BC'] = read_R1_seq[R1['PS1.0-primer']['end_on_read']+1:R1['DBU1-primer']['str_on_read']+1-2]
                            C_seq['DB-BC'] = read_R2_seq[R2['PS2.0-primer']['end_on_read']+1:R2['DBD2-primer']['str_on_read']+1-2] 
                            DB_BC = 1

               
        #DB-lox
        if (('DBloxP-primer' in R1.keys()) and ('DBlox2272-primer' in R2.keys())):     
            if(R1['PS1.0-primer']['end_on_read'] < R1['DBloxP-primer']['str_on_read']):
               if( R1['DBloxP-primer']['str_on_read'] < 50 ):
                   if (R2['PS2.0-primer']['end_on_str']< R2['DBlox2272-primer']['str_on_read']):
                       if(R2['DBlox2272-primer']['str_on_read'] < 50):
               
                            R_seq['DB-lox'] = read_R1_seq[R1['PS1.0-primer']['end_on_read']+1:R1['DBloxP-primer']['str_on_read']+1-2]
                            C_seq['DB-lox'] = read_R2_seq[R2['PS2.0-primer']['end_on_read']+1:R2['DBlox2272-primer']['str_on_read']+1-2] 
                            DB_lox = 1
        #AD-BC
        if( ('ADU1-primer' in R1.keys()) and ('ADD2-primer' in R2.keys()) ):
            if( (R1['PS1.0-primer']['end_on_read'] < R1['ADU1-primer']['str_on_read'])):
               if (R1['ADU1-primer']['str_on_read'] < 50):
                   if (R2['PS2.0-primer']['end_on_read'] < R2['ADD2-primer']['str_on_read']):
                        if (R2['ADD2-primer']['str_on_read'] < 50):

                            R_seq['AD-BC'] = read_R1_seq[R1['PS1.0-primer']['end_on_read']+1:R1['ADU1-primer']['str_on_read']+1-2]
                            C_seq['AD-BC'] = read_R2_seq[R2['PS2.0-primer']['end_on_read']+1:R2['ADD2-primer']['str_on_read']+1-2] 
                            AD_BC = 1

               
        #AD-lox
        if ('ADloxP-primer' in R1.keys()) and ('ADlox2272-primer' in R2.keys())  :   
            if(R1['PS1.0-primer']['end_on_read'] < R1['ADloxP-primer']['str_on_read']):
               if ( R1['ADloxP-primer']['str_on_read'] < 50 ):
                   if (R2['PS2.0-primer']['end_on_str']< R2['ADlox2272-primer']['str_on_read']):
                       if (R2['ADlox2272-primer']['str_on_read'] < 50):
               
                            R_seq['AD-lox'] = read_R1_seq[R1['PS1.0-primer']['end_on_read']+1:R1['ADloxP-primer']['str_on_read']+1-2]
                            C_seq['AD-lox'] = read_R2_seq[R2['PS2.0-primer']['end_on_read']+1:R2['ADlox2272-primer']['str_on_read']+1-2] 
                            AD_lox = 1


        # Specific category assignment?
        category = 0
        R_seq = 0
        C_seq = 0

        if(DB_BC + DB_lox + AD_BC + AD_lox == 1):
            if(DB_BC==1):
                category = 'DB-BC'
            elif(DB_lox==1):
                category = 'DB-lox'
            elif(AD_BC==1):
                category = 'AD-BC'
            elif(AD_lox==1):
                category = 'AD-lox'


        R_SEQ = R_seq[category]
        C_SEQ = C_seq[category]

               
    return (category,P_SEQ1,P_SEQ2,R_SEQ,C_SEQ)



In [57]:
# Defining small functions for calling barcode category and tags index.

def rv_comp(seq):
    bases = {"A":"T","T":"A","G":"C","C":"G","N":"N"}
    seq_l = list(seq)
    rv_comp = [bases[base] for base in seq_l]
    rv_comp.reverse()
    rv_comp
    return ("").join(rv_comp)


def BC_match(R1,R2):
    r1 = R1
    r2 = rv_comp(R2)        

    rel = {}
    for i in range(0,length(r1)):
        n1 = r1[i]
        for j in range(0,length(r2)):
            n2 = r2[j]
            if(n1 == n2):
                diff = i-j
                try:
                    rel[diff] +=1
                except KeyError:
                    rel[diff] = 1

    max_match_diff = max(rel.iteritems(), key=operator.itemgetter(1))[0]                 
                    
    if rel[max_match_diff] >= 18:
        diff  = max_match_diff
        count = rel[max_match_diff]
        
        r1_bc = r1
        r2_bc = r2[diff:]
           
        for i in range(0,len(r1)):
    
        
            nuc_r1 = r1_bc[i]
            if (len(r2_bc)+1) > i:
                nuc_r2 = r2_bc[i]
                if nuc_r1 == nuc_r2:
                    n = nuc_r1
                elif((nuc_r1=="N") and (nuc_r2 != "N")):
                    n = nuc_r2
                elif((nuc_r1!="N") and (nuc_r2 == "N")):
                    n = nuc_r2
                else:
                    n = "N"
            else:
                n = nuc_r1
            
            tag.append(n)

    return ("").join(tag)

def tag_calling(bar2num,seq):
    count = {}
    tag_hits = []
    for tag in bar2num:
    
        for k in range(0,len(tag)):
            k_nuc = tag[k]
            
            for i in tange(0,length(seq)):
                i_nuc = seq[i]

                rel = k-i
                if(k_nuc == i_nuc):
                    try:
                        count[rel] += 1
                    except KeyError:
                        count[rel] = 1
                else:
                    try:
                        count[rel] += 0
                    except KeyError:
                        count[rel] = 0
                
                

        max_match_rel = max(count.iteritems(), key=operator.itemgetter(1))[0]
        tag_hits.append(tag,max_match_rel,count[max_match_rel])
   

    target_hits.sort(key=lambda x: x[2])
    

    if(target_hits[0][2] == target_hits[1][2]):
        val = 0
    elif(target_hits[0][2]>5): 
        called_bar = bar2num[target_hits[0][0]]
        val = called_bar
    else:
        val = 0

    return val

In [58]:
# Definfing function for analysing status of each element within barcode cassette.
def analyze_stat(category,R1,R2,seq_DB):
    stat = {"R1":{},"R2":{}}
    
    if category == "DB-BC":
        
        # Setting default status 
        stat["R1"]['DBU1-primer']  = 'there'
        stat["R1"]['cDBU2-primer'] = 'absent'
        stat["R1"]['UPTAG-frag']        = 'absent'
        stat["R1"]['lox2272']      = 'absent'

        stat["R2"]['DBD2-primer']  = 'there'
        stat["R2"]['cDBD1-primer'] = 'absent'
        stat["R2"]['cDNTAG-frag']       = 'absent'


        ##### Existence check; UPTAG
        if( 'cDBU2-primer' in R1.keys()):
            if( R1['DBU1-primer']['end_on_read'] < R1['cDBU2-primer']['str_on_read']):
                TAG_str = R1['DBU1-primer']['end_on_read']+1 -1
                TAG_end = R1['cDBU2-primer']['str_on_read']-1 -1

                stat["R1"]['cDBU2-primer'] = 'there'
                stat["R1"]['UPTAG-frag']        =  R1["seq"][TAG_str:TAG_end]



        ##### Existence check; lox2272
        if('cDBU2-primer'in R1.keys()):
            if('lox2272' in R1.keys()):
                if(R1['cDBU2-primer']['end_on_read'] < R1['lox2272']['str_on_read']):
                    offset = R1['lox2272']['str_on_read'] - R1['cDBU2-primer']['end_on_read'] -1
                    stat["R1"]['lox2272'] = 'there;offset=%d'%(offset)

        ##### Existence check; DNTAG
        if('cDBD1-primer' in R2.keys()):
            if(R2['DBD2-primer']['end_on_read'] < R2['cDBD1-primer']['str_on_read']):
                TAG_str = R2['DBD2-primer']['emnd_on_read'] +1 -1
                TAG_end = R2['cDBD1-primer']['str_on_read']-1 -1

                stat["R2"]['cDBD1-primer'] = 'there'
                stat["R2"]['cDNTAG-frag']       = R2["seq"][TAG_str:TAG_end]   


        ##### Sequence match check
        for read in stat:
            for primer in stat[read]:
                s =  stat[read][primer].split(";")
                if s[0] == 'there':
                    read_seq = R1['seq'][R1[primer]["str_on_read"]:R1[primer]["end_on_read"]] 
                    if read_seq == seq_DB[primer]:
                        stat[read][primer].replace("there","match")
                    else:
                        stat[read][primer].replace("there","no-match")
    
    if category =="DB-lox":
        # Setting default status 
        stat["R1"]['DBloxP-primer']   = 'there'
        stat["R1"]['cloxP']           = 'absent'
        stat["R1"]['DBU1-primer']     = 'absent'
        stat["R1"]['UPTAG-frag']      = 'absent'                        
        stat["R1"]['cDBU2-primer']    = 'absent'

        stat["R2"]['DBlox2272-primer']= 'there'
        stat["R2"]['clox2272']        = 'absent'
        stat["R2"]['DBU2-primer']= 'there'
        stat["R2"]['cUPTAG-frag']= 'there'
        stat["R2"]['cDBU1-primer']= 'there'
        
        ##### Existence check; cLoxP
        if( 'cloxP' in R1.keys()):
            if( R1['DBloxP-primer']['end_on_read'] < R1['cloxP']['str_on_read']):
                offset = R1['cloxP']['str_on_read'] - R1['DBloxP-primer']['end_on_read'] -1
                stat["R1"]['cloxP'] = 'there;offset=%d'%(offset)

        #### Existence check; DBU1-primer
        if( 'cloxP' in R1.keys()):
            if( 'DBU1-primer' in R1.keys()):    
                if( R1['cloxP']['end_on_read'] < R1['DBU1-primer']['str_on_read']):
                    offset = R1['DBU1-primer']['str_on_read'] - R1['cloxP']['end_on_read'] -1
                    stat["R1"]['DBU1-primer'] = 'there;offset=%d'%(offset)

        #### Existence check; cDBU2-primer
        if( 'cDBU2-primer' in R1.keys()):
            if( 'DBU1-primer' in R1.keys()):    
                if( R1['DBU1-primer']['end_on_read'] < R1['cDBU2-primer']['str_on_read']):
                    offset = R1['cDBU2-primer']['str_on_read'] - R1['DBU1-primer']['end_on_read'] -1
                    stat["R1"]['cDBU2-primer'] = 'there;offset=%d'%(offset)

                    TAG_str = R1['DBU1-primer']['end_on_read']+1
                    TAG_end = R1['cDBU2-primer']['str_on_read']-1-1
                    stat["R1"]['UPTAG_frag'] = R1["seq"][TAG_str:TAG_end]

        ##### Existence check; clox2272
        if( 'clox2272' in R1.keys()):
            if( R1['cDBlox2272-primer']['end_on_read'] < R1['clox2272']['str_on_read']):
                offset = R1['clox2272']['str_on_read'] - R1['DBlox2272-primer']['end_on_read'] -1
                stat["R1"]['clox2272'] = 'there;offset=%d'%(offset)

        #### Existence check; DBU2-primer
        if( 'clox2272' in R2.keys()):
            if( 'DBU2-primer' in R2.keys()):    
                if( R2['clox2272']['end_on_read'] < R2['DBU2-primer']['str_on_read']):
                    offset = R2['DBU2-primer']['str_on_read'] - R2['clox2272']['end_on_read'] -1
                    stat["R2"]['DBU2-primer'] = 'there;offset=%d'%(offset)

        #### Existence check; cDBU1-primer
        if( 'DBU2-primer' in R2.keys()):
            if( 'cDBU1-primer' in R2.keys()):    
                if( R2['DBU2-primer']['end_on_read'] < R2['cDBU1-primer']['str_on_read']):
                    offset = R2['cDBU1-primer']['str_on_read'] - R2['DBU2-primer']['end_on_read'] -1
                    stat["R2"]['cDBU1-primer'] = 'there;offset=%d'%(offset)

                    TAG_str = R2['DBU2-primer']['end_on_read']+1
                    TAG_end = R2['cDBU1-primer']['str_on_read']-1-1
                    stat["R2"]['UPTAG_frag'] = R2["seq"][TAG_str:TAG_end]                       
                                

        ##### Sequence match check
        for read in stat:
            for primer in stat[read]:
                s =  stat[read][primer].split(";")
                if s[0] == 'there':
                    read_seq = R1['seq'][R1[primer]["str_on_read"]:R1[primer]["end_on_read"]] 
                    if read_seq == seq_DB[primer]:
                        stat[read][primer].replace("there","match")
                    else:
                        stat[read][primer].replace("there","no-match")                      
        
        #### Fragment match 
        if stat["R1"]["UPTAG-frag"] != "absent":
            if stat["R2"]["cUPTAG-frag"] != "absent":
                frag  = stat["R1"]["UPTAG-frag"]
                cfrag = stat["R2"]["cUPTAG-frag"]
                                
                BC    = BC_match(frag,cfrag)
                                
                stat['merged-UPTAG'] = BC          
    
    
                                
    if category == "AD-BC":
        
        # Setting default status 
        stat["R1"]['ADU1-primer']  = 'there'
        stat["R1"]['cADU2-primer'] = 'absent'
        stat["R1"]['UPTAG-frag']   = 'absent'
        
        stat["R2"]['ADD2-primer']  = 'there'
        stat["R2"]['cADD1-primer'] = 'absent'
        stat["R2"]['cDNTAG-frag']  = 'absent'
        stat["R2"]['loxP']         = 'absent'
                                


        ##### Existence check; UPTAG
        if( 'cADU2-primer' in R1.keys()):
            if( R1['ADU1-primer']['end_on_read'] < R1['cADU2-primer']['str_on_read']):
                TAG_str = R1['ADU1-primer']['end_on_read']+1 -1
                TAG_end = R1['cADU2-primer']['str_on_read']-1 -1

                stat["R1"]['cADU2-primer'] = 'there'
                stat["R1"]['UPTAG-frag']        =  R1["seq"][TAG_str:TAG_end]



        ##### Existence check; loxP
        if('cADD1-primer'in R1.keys()):
            if('loxP' in  R1.keys()):
                if(R1['cADD1-primer']['end_on_read'] < R1['loxP']['str_on_read']):
                    offset = R1['loxP']['str_on_read'] - R1['ADD1-primer']['end_on_read'] -1
                    stat["R1"]['loxP'] = 'there;offset=%d'%(offset)

        ##### Existence check; DNTAG
        if('cADD1-primer' in R2.keys()):
            if(R2['ADD2-primer']['end_on_read'] < R2['cADD1-primer']['str_on_read']):
                TAG_str = R2['ADD2-primer']['end_on_read'] +1 -1
                TAG_end = R2['cADD1-primer']['str_on_read']-1 -1

                stat["R2"]['cADD1-primer'] = 'there'
                stat["R2"]['cDNTAG-frag']  = R2["seq"][TAG_str:TAG_end]   


        ##### Sequence match check
        for read in stat:
            for primer in stat[read]:
                s =  stat[read][primer].split(";")
                if s[0] == 'there':
                    read_seq = R1['seq'][R1[primer]["str_on_read"]:R1[primer]["end_on_read"]] 
                    if read_seq == seq_DB[primer]:
                        stat[read][primer].replace("there","match")
                    else:
                        stat[read][primer].replace("there","no-match")
    
    if category =="AD-lox":
        # Setting default status 
        stat["R1"]['ADloxP-primer']   = 'there'
        stat["R1"]['cloxP']           = 'absent'
        stat["R1"]['ADD1-primer']     = 'absent'
        stat["R1"]['DNTAG-frag']      = 'absent'                        
        stat["R1"]['cADD2-primer']    = 'absent'

        stat["R2"]['ADlox2272-primer']= 'there'
        stat["R2"]['clox2272']        = 'absent'
        stat["R2"]['ADD2-primer']= 'there'
        stat["R2"]['cDNTAG-frag']= 'there'
        stat["R2"]['cADD1-primer']= 'there'
        
        ##### Existence check; cLoxP
        if( 'cloxP' in R1.keys()):
            if( R1['ADloxP-primer']['end_on_read'] < R1['cloxP']['str_on_read']):
                offset = R1['cloxP']['str_on_read'] - R1['ADloxP-primer']['end_on_read'] -1
                stat["R1"]['cloxP'] = 'there;offset=%d'%(offset)

        #### Existence check; ADD1-primer
        if( 'cloxP' in R1.keys()):
            if( 'ADD1-primer' in R1.keys()):    
                if( R1['cloxP']['end_on_read'] < R1['ADD1-primer']['str_on_read']):
                    offset = R1['ADD1-primer']['str_on_read'] - R1['cloxP']['end_on_read'] -1
                    stat["R1"]['ADD1-primer'] = 'there;offset=%d'%(offset)

        #### Existence check; cADD2-primer
        if( 'cADD2-primer' in R1.keys()):
            if( 'ADD1-primer' in R1.keys()):    
                if( R1['ADD1-primer']['end_on_read'] < R1['cADD2-primer']['str_on_read']):
                    offset = R1['cADD2-primer']['str_on_read'] - R1['ADD1-primer']['end_on_read'] -1
                    stat["R1"]['cADD2-primer'] = 'there;offset=%d'%(offset)

                    TAG_str = R1['ADD1-primer']['end_on_read']+1
                    TAG_end = R1['cADD2-primer']['str_on_read']-1-1
                    stat["R1"]['UPTAG_frag'] = R1["seq"][TAG_str:TAG_end]

        ##### Existence check; clox2272
        if( 'clox2272' in R1.keys()):
            if( R1['ADlox2272-primer']['end_on_read'] < R1['clox2272']['str_on_read']):
                offset = R1['clox2272']['str_on_read'] - R1['ADlox2272-primer']['end_on_read'] -1
                stat["R1"]['clox2272'] = 'there;offset=%d'%(offset)

        #### Existence check; ADD2-primer
        if( 'clox2272' in R2.keys()):
            if( 'ADD2-primer' in R2.keys()):    
                if( R2['clox2272']['end_on_read'] < R2['ADD2-primer']['str_on_read']):
                    offset = R2['ADD2-primer']['str_on_read'] - R2['clox2272']['end_on_read'] -1
                    stat["R2"]['ADD2-primer'] = 'there;offset=%d'%(offset)

        #### Existence check; cADD1-primer
        if( 'ADD2-primer' in R2.keys()):
            if( 'cADD1-primer' in R2.keys()):    
                if( R2['ADD2-primer']['end_on_read'] < R2['cADD1-primer']['str_on_read']):
                    offset = R2['cADD1-primer']['str_on_read'] - R2['ADD2-primer']['end_on_read'] -1
                    stat["R2"]['cADD1-primer'] = 'there;offset=%d'%(offset)

                    TAG_str = R2['ADD2-primer']['end_on_read']+1
                    TAG_end = R2['cADD1-primer']['str_on_read']-1-1
                    stat["R2"]['UPTAG_frag'] = R2["seq"][TAG_str:TAG_end]                       
                                

        ##### Sequence match check
        for read in stat:
            for primer in stat[read]:
                s =  stat[read][primer].split(";")
                if s[0] == 'there':
                    read_seq = R1['seq'][R1[primer]["str_on_read"]:R1[primer]["end_on_read"]] 
                    if read_seq == seq_DB[primer]:
                        stat[read][primer].replace("there","match")
                    else:
                        stat[read][primer].replace("there","no-match")                      
        
        #### Fragment match 
        if stat["R1"]["UPTAG-frag"] != "absent":
            if stat["R2"]["cUPTAG-frag"] != "absent":
                frag  = stat["R1"]["UPTAG-frag"]
                cfrag = stat["R2"]["cUPTAG-frag"]
                                
                BC    = BC_match(frag,cfrag)
                                
                stat['merged-UPTAG'] = BC          
        
    return stat

In [ ]:
data = {}

f_pairs = []
for f in range(0, len(fasta_files)):
    for f2 in range(0, len(fasta_files)):
        if "R1" in fasta_files[f]:
            if fasta_files[f].replace("R1","R2") == fasta_files[f2]:
                f_pairs.append([fasta_files[f].split(".")[0].split("_")[-1],fasta_files[f],fasta_files[f2]])
f_pairs.sort(key=lambda x: x[0])
#pprint(f_pairs)

for small_file in f_pairs:
    R1_ID = small_file[1].split(".")[0]
    R2_ID = small_file[2].split(".")[0]

    R1_fna = '%s/Data/%s/fragmented_fasta/%s'%(PATH,dir_name,small_file[1])
    R2_fna = '%s/Data/%s/fragmented_fasta/%s'%(PATH,dir_name,small_file[2])

    R1_blast = '%s/Data/%s/blast/out.blast/%s.blast'%(PATH,dir_name,R1_ID)
    R2_blast = '%s/Data/%s/blast/out.blast/%s.blast'%(PATH,dir_name,R2_ID)

    d = parse_blast_output2dict(R1_blast,R2_blast,PATH,dir_name,tag_f)
    pprint(d)
    #print(Stop)

In [ ]:

my $data2;
while(my($read,$value) = each %$data){
    my $R1     = $value->{R1};
    my $R2     = $value->{R2};
    my $seq;
    $seq->{R1} = $seq_reads->{R1}->{$read}->{seq};
    $seq->{R2} = $seq_reads->{R2}->{$read}->{seq};



    # PS1.0 and PS2.0 at the reasonable positions? (end at <40bp)
    # Row and Column priming sites from the reasonable positions? (start at <50bp)
    # Which category? DB-BC / DB-lox / AD-BC / AD-lox?
    # Specific category assignment?
    # Row, Column and Plate tags?

    my ($category,$P1_seq,$P2_seq,$R_seq,$C_seq) = &assign_category($value,$R1,$R2,$seq);
    #print Dumper ($category,$P1_seq,$P2_seq,$R_seq,$C_seq);


    my $P1_num = ($P1_seq && length($P1_seq) < 12)? &barcode_matching($bar2num,$P1_seq):0;
    my $P2_num = ($P2_seq && length($P2_seq) < 12)? &barcode_matching($bar2num,$P2_seq):0;
    my $R_num  = ($R_seq  && length($R_seq)  < 12)? &barcode_matching($bar2num,$R_seq) :0;
    my $C_num  = ($C_seq  && length($C_seq)  < 12)? &barcode_matching($bar2num,$C_seq) :0;

    #############################
    # Process for each category #
    #############################

    if($category && $P1_num*$P2_num*$R_num*$C_num){

	$P1_num = sprintf "%.1f", $P1_num/10;
	$P1_num =~ s/\.//g;
	$P2_num = sprintf "%.1f", $P2_num/10;
	$P2_num =~ s/\.//g;
	$R_num  = sprintf "%.1f", $R_num/10;
	$R_num  =~ s/\.//g;
	$C_num  = sprintf "%.1f", $C_num/10;
	$C_num  =~ s/\.//g;

    my ($P_TAG,$R_TAG,$C_TAG) = ("P$P1_num\-P$P2_num","R$R_num","C$C_num");
	#print "$category\t$P_TAG\t$R_TAG\t$C_TAG\n";


	my $stat;

`
	my ($type1,$type2) = split /\-/, $category;
	if($tag_strct->{$type1}->{$type2}->{$P_TAG}){
	    my $plate_name = $tag_strct->{$type1}->{$type2}->{$P_TAG};
	    $data2->{$category}->{$plate_name}->{$R_TAG}->{$C_TAG}->{$read} = $stat;
	}
    }
}

print Dumper $data2;